# Lab | Web Scraping

Welcome to the IMDb Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the vast sea of data available on IMDb, one of the largest online databases of movie, TV, and celebrity information. As budding data scientists and business analysts, you have been tasked to scrape a specific subset of data from IMDb to assist film production companies in understanding the landscape of highly-rated movies in a defined time period. Your insights will potentially influence the making of the next netflix movie!

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The entertainment industry, being no exception, utilizes data analytics to comprehend market trends, audience preferences, and the performance of films based on various parameters such as director, genre, stars involved, etc. IMDb stands as a goldmine of such data, offering intricate details of almost every movie ever made.

**Task**

Your task is to create a Python script using `BeautifulSoup` and `pandas` to scrape IMDb movie data based on user ratings and release dates. This script should be able to filter movies with ratings above a certain threshold and within a specified date range.

**Expected Outcome**

- A function named `scrape_imdb` that takes four parameters: `title_type`,`user_rating`, `start_date`, and `end_date`.
- The function should return a DataFrame with the following columns:
  - **Movie Nr**: The number representing the movie’s position in the list.
  - **Title**: The title of the movie.
  - **Year**: The year the movie was released.
  - **Rating**: The IMDb rating of the movie.
  - **Runtime (min)**: The duration of the movie in minutes.
  - **Genre**: The genre of the movie.
  - **Description**: A brief description of the movie.
  - **Director**: The director of the movie.
  - **Stars**: The main stars of the movie.
  - **Votes**: The number of votes the movie received.
  - **Gross ($M)**: The gross earnings of the movie in millions of USD.

You will execute this script to scrape data for movies with the Title Type `Feature Film` that have a user rating of `7.5 and above` and were released between `January 1, 1990, and December 31, 1992`.

Remember to experiment with different title types, dates and ratings to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [IMDb Advanced Search](https://www.imdb.com/search/title/)


**Hint**

Your first mission is to familiarize yourself with the IMDb advanced search page. Head over to [IMDb advanced search](https://www.imdb.com/search/title/) and input the following parameters, keeping all other fields to their default values or blank:

- **Title Type**: Feature film
- **Release date**: From 1990 to 1992 (Note: You don't need to specify the day and month)
- **User Rating**: 7.5 to -

Upon searching, you'll land on a page showcasing a list of movies, each displaying vital details such as the title, release year, and crew information. Your task is to scrape this treasure trove of data.

Carefully examine the resulting URL and construct your own URL to include all the necessary parameters for filtering the movies.


---

**Best of luck! Immerse yourself in the world of movies and may the data be with you!**

**Important note**:

In the fast-changing online world, websites often get updates and make changes. When you try this lab, the IMDb website might be different from what we expect.

If you run into problems because of these changes, like new rules or things that stop you from getting data, don't worry! Instead, get creative.

You can choose another website that interests you and is good for scraping data. Websites like Wikipedia or The New York Times are good options. The main goal is still the same: get useful data and learn how to scrape it from a website that you find interesting. It's a chance to practice your web scraping skills and explore a source of information you like.

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions


In [7]:
def convert_to_minutes(duration):
    # Inicializamos las horas y minutos
    hours = 0
    minutes = 0
    
    # Si la duración contiene horas, extraemos el valor antes de 'h'
    if 'h' in duration:
        hours = int(duration.split('h')[0].strip())
    
    # Si la duración contiene minutos, extraemos el valor después de 'h'
    if 'm' in duration:
        minutes = int(duration.split('h')[1].replace('m', '').strip())
    
    # Convertimos todo a minutos (1h = 60 minutos) y lo sumamos
    total_minutes = hours * 60 + minutes
    return total_minutes


def scrape_imdb(title_type, user_rating, start_date, end_date):
    # Construcción de la URL basada en los parámetros proporcionados
    base_url = 'https://www.imdb.com/search/title/?'
    query = f'title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},'
    url = base_url + query

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Referer': 'https://www.google.com'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        driver = webdriver.Chrome()
        driver.get(url)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(5)
        page_source = driver.page_source
        driver.quit()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        movies = soup.find_all('li', class_='ipc-metadata-list-summary-item')
    else:
        print("Error making the HTTP request:", response.status_code)


    movie_nrs = []
    titles = []
    years = []
    ratings = []
    runtimes = []
    genres = []
    descriptions = []
    directors = []
    stars = []
    votes = []
    gross_earnings = []

    for idx, movie in enumerate(movies):
        movie_nrs.append(idx + 1)
        title = movie.find("h3", class_="ipc-title__text")
        title = title.text.lstrip('1234567890').lstrip('.').strip()
        titles.append(title)

        dates = movie.find_all("span", class_="sc-b189961a-8 hCbzGp dli-title-metadata-item")
        
        it=0
        for date in dates:
            if it == 0:
                #year = movie.find("span", class_="sc-b189961a-8 hCbzGp dli-title-metadata-item")
                year = int(date.text)
                years.append(year)
            elif it == 1:
                runtime = convert_to_minutes(date.text)
                runtimes.append(runtime)
            elif it == 2:
                ratings.append(date.text)
            it += 1

        #rating = movie.find('span', class_='sc-b0901df4-0 bXIOoL metacritic-score-box')
        #ratings.append(int(rating.text))

        
        #genre = movie.find('span', class_='ipc-chip__text')
        #url2 = movie.find('a', class_='ipc-title-link-wrapper')
        #response2 = 
        #genres.append(url2)

        description = movie.find('div', class_='ipc-html-content-inner-div').text
        descriptions.append(description)

        #director = movie.find()

        star = movie.find("span", class_="ipc-rating-star--rating")
        star = float(star.text)
        stars.append(star)

        vote = movie.find('span', class_='ipc-rating-star--voteCount').text.strip('\xa0()')
        votes.append(vote)

        #gross
        #gross_tag = movie.find_all('span', attrs={'name': 'nv'})
        #gross_earnings.append(gross_tag)

    df = pd.DataFrame({
        'Movie Nr': movie_nrs,
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': runtimes,
        #'Genre': genres,
        'Description': descriptions,
        #'Director': directors,
        'Stars': stars,
        'Votes': votes,
        #'Gross ($M)': gross_earnings
    })
    return df



In [8]:
df = scrape_imdb('feature', 7.5, '1990-01-01', '1992-12-31')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [14]:
title_type = 'feature'
user_rating = 7.5
start_date = '1990-01-01'
end_date = '1992-12-31'


In [23]:
df

,Movie Nr,Title,Year,Rating,Runtime (min),Description,Stars,Votes
0,1,The Silence of the Lambs,1991,R,118,A young F.B.I. cadet must receive the help of ...,8.6,1.6M
1,2,Terminator 2: Judgment Day,1991,R,137,"A cyborg, identical to the one who failed to k...",8.6,1.2M
2,3,Goodfellas,1990,R,145,The story of Henry Hill and his life in the ma...,8.7,1.3M
3,4,Edward Scissorhands,1990,PG-13,105,The solitary life of an artificial man - who w...,7.9,532K
4,5,Reservoir Dogs,1992,R,99,When a simple jewelry heist goes horribly wron...,8.3,1.1M
5,6,Home Alone,1990,PG,103,"An eight-year-old troublemaker, mistakenly lef...",7.7,662K
6,7,My Cousin Vinny,1992,R,120,Two New Yorkers accused of murder in rural Ala...,7.6,144K
7,8,The Last of the Mohicans,1992,R,112,Three trappers protect the daughters of a Brit...,7.6,191K
8,9,A Few Good Men,1992,R,138,"Lt. Daniel Kaffee, a US military lawyer, defen...",7.7,293K
9,10,Unforgiven,1992,R,130,Retired Old West gunslinger Will Munny relucta...,8.2,442K


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Función para hacer scraping de IMDb
def scrape_imdb(title_type, user_rating, start_date, end_date):
    # Construcción de la URL basada en los parámetros proporcionados
    base_url = 'https://www.imdb.com/search/title/?'
    query = f'title_type={title_type}&release_date={start_date},{end_date}&user_rating={user_rating},'
    url = base_url + query
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Hacer la solicitud a la página web con headers
    response = requests.get(url, headers=headers)
    # Hacer la solicitud a la página web
    #response = requests.get(url)
    
    # Verificar que la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la página: {response.status_code}")
        return None

    # Parsear el HTML de la página con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraer la lista de películas
    movies = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Inicializar listas para almacenar la información
    movie_nrs = []
    titles = []
    years = []
    ratings = []
    runtimes = []
    genres = []
    descriptions = []
    directors = []
    stars = []
    votes = []
    gross_earnings = []

    # Iterar sobre cada película y extraer la información
    for idx, movie in enumerate(movies):
        # Número de la película
        movie_nrs.append(idx + 1)
        
        # Título de la película
        title = movie.h3.a.text
        titles.append(title)
        
        # Año de la película
        year = movie.h3.find('span', class_='lister-item-year').text
        years.append(year.strip('()'))
        
        # Calificación de la película
        rating = movie.find('div', class_='inline-block ratings-imdb-rating')
        if rating:
            ratings.append(float(rating.strong.text))
        else:
            ratings.append(None)
        
        # Duración de la película
        runtime = movie.find('span', class_='runtime')
        if runtime:
            runtimes.append(int(runtime.text.replace(' min', '')))
        else:
            runtimes.append(None)
        
        # Género de la película
        genre = movie.find('span', class_='genre')
        if genre:
            genres.append(genre.text.strip())
        else:
            genres.append(None)
        
        # Descripción de la película
        description = movie.find_all('p', class_='text-muted')[1].text.strip()
        descriptions.append(description)
        
        # Director y estrellas de la película
        crew = movie.find('p', class_='').text.strip().split('|')
        if len(crew) > 1:
            director_stars = crew[0].replace('Director:', '').strip()
            directors.append(director_stars)
        else:
            directors.append(None)

        stars_list = movie.find_all('a', href=True)
        stars.append(", ".join([star.text for star in stars_list[-4:-1]]))
        
        # Votos de la película
        votes_tag = movie.find('span', attrs={'name': 'nv'})
        if votes_tag:
            votes.append(int(votes_tag['data-value'].replace(',', '')))
        else:
            votes.append(None)
        
        # Ingresos brutos de la película (Gross Earnings)
        gross_tag = movie.find_all('span', attrs={'name': 'nv'})
        if len(gross_tag) > 1:
            gross = gross_tag[1].text.strip('$M')
            gross_earnings.append(float(gross.replace(',', '')) if gross != ' ' else None)
        else:
            gross_earnings.append(None)
    
    # Crear DataFrame con la información extraída
    df = pd.DataFrame({
        'Movie Nr': movie_nrs,
        'Title': titles,
        'Year': years,
        'Rating': ratings,
        'Runtime (min)': runtimes,
        'Genre': genres,
        'Description': descriptions,
        'Director': directors,
        'Stars': stars,
        'Votes': votes,
        'Gross ($M)': gross_earnings
    })
    
    return df

# Probar la función con los parámetros dados
df = scrape_imdb('feature', 7.5, '1990-01-01', '1992-12-31')

# Mostrar los primeros resultados
print(df.head())


Empty DataFrame
Columns: [Movie Nr, Title, Year, Rating, Runtime (min), Genre, Description, Director, Stars, Votes, Gross ($M)]
Index: []


In [4]:
response.headers

{'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'Server', 'Date': 'Mon, 09 Sep 2024 17:50:21 GMT', 'Strict-Transport-Security': 'max-age=47474747; includeSubDomains; preload', 'Vary': 'Content-Type,Accept-Encoding,User-Agent', 'x-amz-rid': 'N8SDPE4MA8W5ZCDMFAT1', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 359bc1d5577c7c49388c1dfe062fa074.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'MAD56-P3', 'Alt-Svc': 'h3=":443"; ma=86400', 'X-Amz-Cf-Id': 't-s9cdJrwJhM1Pzhr4UgUSj2ArTbxXa2USbRWAKYbCp0sHRriGx6hA=='}

In [ ]:
def scrape_imdb(title_type, user_rating, start_date, end_date):
    

## BONUS

The search results span multiple pages, housing a total of 631 movies in our example with each page displaying 50 movies at most. To scrape data seamlessly from all pages, you'll need to dive deep into the structure of the URLs generated with each "Next" click.

Take a close look at the following URLs:
- First page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,
  ```
- Second page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
  ```
- Third page:
  ```
  https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
  ```

You should notice a pattern. There is a `start` parameter incrementing by 50 with each page, paired with a constant `ref_` parameter holding the value "adv_nxt".

Modify your script so it's capable of iterating over all available pages to fetch data on all the 631 movies (631 is the total number of movies in the proposed example).

In [ ]:
# Your solution goes here